<a href="https://colab.research.google.com/github/simonprudhomme/optimization_with_ortools/blob/main/or_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Automated Reshuffling

In [1]:
# install ortools
!pip install ortools

In [2]:
# import packages
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np
import json

In [3]:
# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver('factory', problem_type=pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [4]:
demand_per_fsa = {'H2H': {'parcels' : np.random.randint(1,1000),
                          'distance_station1': np.random.randint(1,75),
                          'distance_station2': np.random.randint(1,75)
                          },
                  'H3H': {'parcels' : np.random.randint(1,1000),
                          'distance_station1': np.random.randint(1,75),
                          'distance_station2': np.random.randint(1,75)
                          },
                  'H4H': {'parcels' : np.random.randint(1,1000),
                          'distance_station1': np.random.randint(1,75),
                          'distance_station2': np.random.randint(1,75)
                          },
                  'H5H': {'parcels' : np.random.randint(1,1000),
                          'distance_station1': np.random.randint(1,75),
                          'distance_station2': np.random.randint(1,75)
                          },
                  'H6H': {'parcels' : np.random.randint(1,1000),
                          'distance_station1': np.random.randint(1,75),
                          'distance_station2': np.random.randint(1,75)
                          },
                  'H7H': {'parcels' : np.random.randint(1,1000),
                          'distance_station1': np.random.randint(1,75),
                          'distance_station2': np.random.randint(1,75)
                          },
                  'H8H': {'parcels' : np.random.randint(1,1000),
                          'distance_station1': np.random.randint(1,75),
                          'distance_station2': np.random.randint(1,75)
                          },
                  'H9H': {'parcels' : np.random.randint(1,1000),
                          'distance_station1': np.random.randint(1,75),
                          'distance_station2': np.random.randint(1,75)
                          }
                  }


stations = {'station_1':{'capacity': 4000},
            'station_2':{'capacity': 2000}
            }

for i in [stations,demand_per_fsa]:
  print(json.dumps(i, indent = 4))

{
    "station_1": {
        "capacity": 4000
    },
    "station_2": {
        "capacity": 2000
    }
}
{
    "H2H": {
        "parcels": 475,
        "distance_station1": 7,
        "distance_station2": 70
    },
    "H3H": {
        "parcels": 602,
        "distance_station1": 66,
        "distance_station2": 3
    },
    "H4H": {
        "parcels": 628,
        "distance_station1": 18,
        "distance_station2": 44
    },
    "H5H": {
        "parcels": 719,
        "distance_station1": 43,
        "distance_station2": 45
    },
    "H6H": {
        "parcels": 953,
        "distance_station1": 5,
        "distance_station2": 51
    },
    "H7H": {
        "parcels": 748,
        "distance_station1": 66,
        "distance_station2": 53
    },
    "H8H": {
        "parcels": 310,
        "distance_station1": 50,
        "distance_station2": 71
    },
    "H9H": {
        "parcels": 663,
        "distance_station1": 41,
        "distance_station2": 8
    }
}


In [5]:
# Create the variables

options = {}
for fsa in demand_per_fsa.keys():
  option_tmp = {}
  for station in stations.keys():
    option_tmp[f'{fsa}_{station}'] = solver.IntVar(0, 1, f'{fsa}_{station}')
  options[f'{fsa}']=option_tmp

print('Number of variables  =', solver.NumVariables())

Number of variables  = 16


In [8]:
# Create the constraints

# Allocate FSA to one station only
for fsa in options.keys():
  solver.Add(options[fsa][f'{fsa}_station_1'] + options[fsa][f'{fsa}_station_2'] == 1)

# Allocate all FSAs 
solver.Add(solver.Sum([(options[fsa][f'{fsa}_station_1'] * demand_per_fsa[fsa]['parcels'] + options[fsa][f'{fsa}_station_2'] * demand_per_fsa[fsa]['parcels']) for fsa in demand_per_fsa.keys()])==sum([demand_per_fsa[fsa]['parcels'] for fsa in demand_per_fsa.keys()]))

# Respect stations Capacity
solver.Add(solver.Sum([options[fsa][f'{fsa}_station_1'] * demand_per_fsa[fsa]['parcels'] for fsa in demand_per_fsa.keys()]) <= stations['station_1']['capacity'])
solver.Add(solver.Sum([options[fsa][f'{fsa}_station_2'] * demand_per_fsa[fsa]['parcels'] for fsa in demand_per_fsa.keys()]) <= stations['station_2']['capacity'])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 11


In [9]:
# Create the objective function : reduce distance from FSA to STATIONS while respecting stations capacity

solver.Minimize(solver.Sum([options[fsa][f'{fsa}_station_1'] * demand_per_fsa[fsa]['distance_station1'] +  options[fsa][f'{fsa}_station_2'] * demand_per_fsa[fsa]['distance_station2'] for fsa in demand_per_fsa.keys()]))

In [10]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    station_1 = 0
    station_2 = 0
    for fsa in demand_per_fsa.keys():
      if options[fsa][f'{fsa}_station_1'].solution_value() >= 1.:
        print(f'{fsa} allocated to station 1')
        station_1 += demand_per_fsa[fsa]['parcels']
      else:
        print(f'{fsa} allocated to station 2')
        station_2 += demand_per_fsa[fsa]['parcels']
    print('Total parcels station1:', station_1)
    print('Total parcels station2:', station_2)

else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 200.0
H2H allocated to station 1
H3H allocated to station 2
H4H allocated to station 1
H5H allocated to station 1
H6H allocated to station 1
H7H allocated to station 1
H8H allocated to station 1
H9H allocated to station 2
Total parcels station1: 3833
Total parcels station2: 1265


Suggested next steps 
- try to find optimal FSAs allocation for QC region